In [1]:
pip install pytorch_lightning

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pytorch_lightning.utilities.types import OptimizerLRScheduler

In [3]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForQuestionAnswering, BertTokenizer
from torch.optim import AdamW
import pandas as pd

2026-01-30 16:24:06.522722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769790246.920778      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769790247.041619      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769790248.100432      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769790248.100473      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769790248.100477      55 computation_placer.cc:177] computation placer alr

In [5]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
import gdown
import os


folder_id = '1Hkkt5pzvpHXk_dHCNl62HOpnvzfuy1RG'
url = f'https://drive.google.com/drive/folders/{folder_id}'


output_dir = '/kaggle/working/my_dataset'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

gdown.download_folder(url, output=output_dir, quiet=False, remaining_ok=True)


print("Files in dataset folder:", os.listdir(output_dir))

Retrieving folder contents


Processing file 1aru0XD4-QEXtObYk92hCU5eYO4AmrSeo qna_test.csv
Processing file 1rqRraaaDduNElM3i9W9IS6iiv81QXWZc qna_train.csv
Processing file 13dYYmfC4mZQEL5rdLAg4xRxJt9OXJKLq QnA_with_llm.ipynb


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1aru0XD4-QEXtObYk92hCU5eYO4AmrSeo
To: /kaggle/working/my_dataset/qna_test.csv
100%|██████████| 10.2M/10.2M [00:00<00:00, 37.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rqRraaaDduNElM3i9W9IS6iiv81QXWZc
To: /kaggle/working/my_dataset/qna_train.csv
100%|██████████| 77.3M/77.3M [00:01<00:00, 64.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=13dYYmfC4mZQEL5rdLAg4xRxJt9OXJKLq
To: /kaggle/working/my_dataset/QnA_with_llm.ipynb
100%|██████████| 118k/118k [00:00<00:00, 72.3MB/s]

Files in dataset folder: ['qna_test.csv', 'qna_train.csv', 'QnA_with_llm.ipynb']



Download completed


In [10]:
class QnADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answers = eval(self.df.iloc[idx]['answers'])['text'][0]
        answer_start = eval(self.df.iloc[idx]['answers'])['answer_start'][0]

        try:
            tokens = self.tokenizer.encode_plus(
                context,
                question,
                add_special_tokens=True,
                max_length=512,
                truncation=True,
                padding='max_length',
                return_tensors='pt',
                return_offsets_mapping=True
                )
        except Exception as e:
            tokens = {
                'input_ids': torch.zeros(512, dtype=torch.long),
                'attention_mask': torch.zeros(512, dtype=torch.long),
                'offset_mapping': torch.zeros((512, 2), dtype=torch.long),
            }

        input_ids = tokens['input_ids'].squeeze()
        attention_mask = tokens['attention_mask'].squeeze()
        offset_mapping = tokens['offset_mapping'].squeeze()

        start_position = 0
        end_position = 0
        answer_end = answer_start + len(answers)

        for i, (start_char, end_char) in enumerate(offset_mapping):
            if start_char <= answer_start < end_char:
                start_position = i
            if start_char < answer_end <= end_char:
                end_position = i
                break

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': torch.tensor([start_position], dtype=torch.long),
            'end_positions': torch.tensor([end_position], dtype=torch.long),
        }


In [11]:
class QnADataModule(pl.LightningDataModule):
    def __init__(self, train_data_path, test_data_path, tokenizer):
        super().__init__()
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.tokenizer = tokenizer

    def setup(self, stage=None):
        train_df = pd.read_csv(self.train_data_path)
        test_df = pd.read_csv(self.test_data_path)
        self.train_ds = QnADataset(train_df, self.tokenizer)
        self.test_ds = QnADataset(test_df, self.tokenizer)

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=4,
            shuffle=True,
        )
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=4,
            shuffle=False,
        )

In [12]:
from transformers import AutoModelForQuestionAnswering


class QnAModel(pl.LightningModule):
    def __init__(self, model_name='deepset/roberta-base-squad2'):
        super().__init__()
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, start_positions, end_positions):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions,
        )
        return outputs
    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = self.forward(input_ids,attention_mask,start_positions,end_positions)

        loss = outputs.loss
        self.log('train_loss', loss, prog_bar=True)
        return loss
    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = self.forward(input_ids,attention_mask,start_positions,end_positions)

        loss = outputs.loss
        self.log('test_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return AdamW(params=self.parameters(), lr=5e-5, eps=1e-8)

In [13]:
from transformers import AutoTokenizer

model_name = 'deepset/roberta-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = QnAModel()
ds_module = QnADataModule(
    train_data_path="/kaggle/working/my_dataset/qna_train.csv",
    test_data_path="/kaggle/working/my_dataset/qna_test.csv",
    tokenizer=tokenizer
)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [14]:
trainer = pl.Trainer(
    max_steps=500
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# trainer.fit(model, ds_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┓
┃   ┃ Name  ┃ Type                        ┃ Params ┃ Mode ┃ FLOPs ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━┩
│ 0 │ model │ RobertaForQuestionAnswering │  124 M │ eval │     0 │
└───┴───────┴─────────────────────────────┴────────┴──────┴───────┘

Trainable params: 124 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 124 M                                                                                                
Total estimated model params size (MB): 496                                                                        
Modules in train mode: 0                                                                                           
Modules in eval mode: 227                                                                                          
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:534: Found 227 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=500` reached.


In [ ]:
# trainer.test(model=model, datamodule=ds_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.2798359394073486     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.2798359394073486}]

In [15]:
pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.2/789.2 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-

In [16]:
import mlflow
import mlflow.transformers
from mlflow.models.signature import infer_signature
import shutil

In [17]:


import os


mlflow.set_tracking_uri("file:///kaggle/working/mlruns")

mlflow.set_experiment(experiment_name="QnA_with_LLM")

if not os.path.exists("/kaggle/working/mlruns"):
    os.makedirs("/kaggle/working/mlruns")


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/30 16:40:22 INFO mlflow.tracking.fluent: Experiment with name 'QnA_with_LLM' does not exist. Creating a new experiment.


In [19]:
LEARNING_RATE = 5e-5
BATCH_SIZE = 4
MAX_STEPS = 500

In [20]:
import os
ARTIFACT_FOLDER_NAME = "qa_model"
SOURCE_CODE_PATH = '/kaggle/working/my_dataset/QnA_with_llm.ipynb'
SOURCE_CODE_ARTIFACT = 'trainer.ipynb'

In [22]:
# mlflow.end_run()
with mlflow.start_run() as run:


    # Log hyperparameters
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("max_steps", MAX_STEPS)


    # Initialize model and datamodule
    model = QnAModel(model_name='deepset/roberta-base-squad2')
    ds_module = QnADataModule(
        train_data_path="/kaggle/working/my_dataset/qna_train.csv",
        test_data_path="/kaggle/working/my_dataset/qna_test.csv",
        tokenizer=tokenizer
    )
    ds_module.setup()
    trainer = pl.Trainer(
        max_steps=MAX_STEPS,
     )

    trainer.fit(
        model=model,
        train_dataloaders=ds_module.train_dataloader(),

    )

    score = trainer.test(
        model=model,
        dataloaders=ds_module.test_dataloader()
     )
    test_loss = score[0]["test_loss"]
    mlflow.log_metric("test_loss", test_loss)

    temp_save_dir = "/kaggle/working/temp_model_files"

    # Clean up any old temp files from previous failed runs
    if os.path.exists(temp_save_dir):
        shutil.rmtree(temp_save_dir)
    os.makedirs(temp_save_dir)

    model.model.save_pretrained(temp_save_dir)
    tokenizer.save_pretrained(temp_save_dir)

    mlflow.log_artifacts(temp_save_dir, artifact_path=ARTIFACT_FOLDER_NAME)


    if os.path.exists(SOURCE_CODE_PATH):
        shutil.copyfile(SOURCE_CODE_PATH, SOURCE_CODE_ARTIFACT)
        mlflow.log_artifact(SOURCE_CODE_ARTIFACT)
        print("✅ Source code logged.")
    else:
        print(f"⚠️ Note: {SOURCE_CODE_PATH} not found. skipping notebook log.")

    # Capture IDs for verification
    run_id = run.info.run_id
    exp_id = run.info.experiment_id

mlflow.end_run()
import time
time.sleep(10)

final_path = f"/kaggle/working/mlruns/{exp_id}/{run_id}/artifacts/{ARTIFACT_FOLDER_NAME}"

if os.path.exists(final_path):
    files = os.listdir(final_path)
    print(f"✅ SUCCESS! Artifacts found at: {final_path}")
    print(f"Files found: {files}")
    if "model.safetensors" in files or "pytorch_model.bin" in files:
        print("💎 PROOF: Weights are physically on disk.")
else:
    print(f"❌ ERROR: Artifacts are still not in {final_path}")
    print("Checking if mlruns exists at all:")
    





Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┓
┃   ┃ Name  ┃ Type                        ┃ Params ┃ Mode ┃ FLOPs ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━┩
│ 0 │ model │ RobertaForQuestionAnswering │  124 M │ eval │     0 │
└───┴───────┴─────────────────────────────┴────────┴──────┴───────┘

Trainable params: 124 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 124 M                                                                                                
Total estimated model params size (MB): 496                                                                        
Modules in train mode: 0                                                                                           
Modules in eval mode: 227                                                                                          
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:534: Found 227 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.
`Trainer.fit` stopped: `max_steps=500` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.489785671234131     │
└───────────────────────────┴───────────────────────────┘

✅ Source code logged.
✅ SUCCESS! Artifacts found at: /kaggle/working/mlruns/706558397354895962/0f4ce9983d6a404081fbdadf04b46adb/artifacts/qa_model
Files found: ['vocab.json', 'tokenizer.json', 'config.json', 'merges.txt', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json']
💎 PROOF: Weights are physically on disk. You are ready to zip!


In [23]:
import shutil
from IPython.display import FileLink


artifact_src = '/kaggle/working/mlruns/706558397354895962/0f4ce9983d6a404081fbdadf04b46adb/artifacts/qa_model'


shutil.make_archive('qa_weights', 'zip', artifact_src)


FileLink(r'qa_weights.zip')

/kaggle/working/qa_weights.zip

In [24]:
!pip install pyngrok --quiet
from pyngrok import ngrok
import os

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok
import os


!fuser -k 5000/tcp
ngrok.kill()


user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("ngrok")
ngrok.set_auth_token(secret_value_0)


get_ipython().system_raw("mlflow ui --port 5000 --host 0.0.0.0 --allowed-hosts '*' &")

# 4. Create the tunnel
try:
    public_url = ngrok.connect(5000)
    print("\n" + "="*50)
    print(f"🚀 FIXED LINK: {public_url.public_url}")
    print("="*50)
except Exception as e:
    print(f"❌ Connection failed: {e}")

5000/tcp:              563   565   566   567   568


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



🚀 FIXED LINK: https://nonreductive-noncognizably-frida.ngrok-free.dev
Backend store URI not provided. Using ./mlruns
Registry store URI not provided. Using backend store URI.


/usr/local/lib/python3.12/dist-packages/mlflow/server/handlers.py:310: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, artifact_uri)
/usr/local/lib/python3.12/dist-packages/mlflow/server/handlers.py:339: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be 

INFO:     103.150.255.236:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/js/main.9e6a97ae.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/css/main.280d6c90.css HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/TelemetryLogger.telemetry-worker.e586432cbf500042667b.worker.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/js/783.b107ace8.chunk.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /ajax-api/3.0/mlflow/assistant/config HTTP/1.1" 403 Forbidden
INFO:     103.150.255.236:0 - "GET /static-files/static/js/3528.18b15e38.chunk.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/js/3617.f09cade8.chunk.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/css/547.26533251.chunk.css HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /static-files/static/js/4205.1aba6fbf.chunk.js HTTP/1.1" 200 OK
INFO:     103.150.255.236:0 - "GET /

...

INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=run_id%3D%270f4ce9983d6a404081fbdadf04b46adb%27 HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%250f4ce9983d6a404081fbdadf04b46adb%25%22 HTTP/1.1" 200 OK


/usr/local/lib/python3.12/dist-packages/mlflow/server/handlers.py:339: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri)


INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%250f4ce9983d6a404081fbdadf04b46adb%25%22 HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/2.0/mlflow/experiments/search-datasets HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/traces?experiment_ids=706558397354895962&order_by=timestamp_ms%20DESC&max_results=1&filter= HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "POST /ajax-api/3.0/mlflow/ui-telemetry HTTP/1.1" 200 OK
INFO:     103.

2026/01/30 17:47:03 ERROR mlflow.server.handlers: Error in _search_evaluation_datasets_handler: search_datasets is not supported with FileStore. This feature requires a SQL-based tracking backend (e.g., SQLite, PostgreSQL, MySQL). Please configure MLflow with a SQL backend using --backend-store-uri. For SQLite setup instructions, see: https://mlflow.org/docs/latest/self-hosting/architecture/tracking-server/#configure-server. Set MLFLOW_LOGGING_LEVEL=DEBUG for traceback.


INFO:     103.150.255.237:0 - "POST /ajax-api/3.0/mlflow/datasets/search HTTP/1.1" 500 Internal Server Error
INFO:     103.150.255.237:0 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     103.150.255.237:0 - "GET /ajax-api/2.0/mlflow/artifacts/list?run_uuid=0f4ce9983d6a404081fbdadf04b46adb HTTP/1.1" 200 OK
INFO:     103.150.255.237:0 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     103.150.255.237:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%250f4ce9983d6a404081fbdadf04b46adb%25%22 HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%250f4ce9983d6a404081fbdadf04b46adb%25%22 HTTP/1.1" 200 OK
INFO:     103.150.255.235:0 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is